In [5]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt; plt.style.use('ggplot')
from scipy import stats

In [2]:
pd.set_option('display.max_columns', None)

In [2]:
data = pd.read_csv('/Users/willfitzhugh/Desktop/Coding/Supply-Chain-Analysis/Data/Interim/Ready_For_Fraud_Featuers.csv')
copy = data.copy()

In [168]:
data = copy.copy()

In [93]:
#feature engineering to deal with catagorical variables with high cardinality:
#flag cities and states with high fraud rate, and those with no recorded fraud
#same with product names
#cols to engineer: ProductCategory, Customer State, Customer City, ProductDepartment, Order City, Order Country, Order region,
#Order state, Product Name

#cols to remove: Customer Id, Latitude, Longitude, Order date, Ship date, Order Id, Order status, Customer Name,
#order year, real ship days, delivery status, late_delivery_risk, order Status, order year and probably one or more 
#of [OrderProfit, OrderSales, Order Item Discount, Order Item Discount Rate,Order Item Profit Ratio, product price]

#cols to one hot encode: PaymentType, Customer Country, Customer Segment, Market, shipping mode, Order DOW

In [3]:
dums = pd.get_dummies(data[['PaymentType', 'Customer Country', 'Customer Segment', 'Market', 'Shipping Mode',
                            'OrderDOW']])

data = pd.concat([data,dums], axis=1).drop(columns=['PaymentType', 'Customer Country', 'Customer Segment', 'Market',
                                             'Shipping Mode','OrderDOW'])

In [6]:
#have different approach with columns with high cardinality. order city has much higher cardinality than order region
#and should be binned differently
'''
want to keep bins from over fitting to highly cardinal data. I want to avoid marking a city as extremely high risk 
just because it has one fraud order, and zero non-fraud orders. I still want to mark it has higher risk, but marking
it as garaunteed fraud might cause data leakage and over fitting.
'''
bin_cols = ['ProductCategory', 'Customer State', 'Customer City', 'ProductDepartment', 'Order City', 'Order Country',
            'Order Region', 'Order State', 'Product Name']

for i in bin_cols: 

    grouped = data.groupby(i).mean()[['IsFraud']].sort_values('IsFraud', ascending=False).reset_index()
    binned = stats.binned_statistic(x = grouped['IsFraud'], values = grouped['IsFraud'], bins = [0, .001, .015, .02, .03, .045, .2, 1])

    grouped['bin'] = binned[2]
    
    bin_map = grouped[[i,'bin']].set_index(i).to_dict()['bin']
    
    data[i] = data[i].map( bin_map )
    
    
'''
This mapping strategy/ target encoding in general presents a problem. I may not have data on an order city, so 
my model will not be able to handle the order. I could create another model, one that doesn't depend on order city
and use that model if the input order has an order city that didn't appear in the train dataset.
'''

"\nThis mapping strategy/ target encoding in general presents a problem. I may not have data on an order city, so \nmy model will not be able to handle the order. I could create another model, one that doesn't depend on order city\nand use that model if the input order has an order city that didn't appear in the train dataset.\n"

In [7]:
data = data.drop(columns = ['Customer Id','Latitude','Longitude','OrderDate','ShipDate','Order Id','Order Status', 
                  'CustomerName', 'OrderYear', 'RealShippingDays', 'Delivery Status', 'Late_delivery_risk',
                  'Order Status','OrderHour','OrderMonth','Order Item Discount','OrderProfit','Order Item Quantity'])


In [8]:
data.IsFraud = data.IsFraud.map({True:1,False:0})

In [11]:
data.to_csv('/Users/willfitzhugh/Desktop/Coding/Supply-Chain-Analysis/Data/Model/FraudData_1.0.csv', index=False)